In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from transformers import BertTokenizer
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/bertmodel/new')
from bert import minBert
from bert import EncoderLayer, SelfAttention, FeedForward, BaseAttention
from positional_embedding import PositionalEmbedding
sys.path.append('/content/drive/MyDrive/Colab Notebooks/bertmodel/contrastive_learning')
from contrastive_learning import SupervisedContrastiveModel

In [ ]:
tokenizer = BertTokenizer.from_pretrained("google-bert/bert-base-uncased")

In [ ]:
min_bert_layer = tf.keras.models.load_model(
    '/content/drive/MyDrive/Colab Notebooks/bertmodel/min_bert_layer.keras',
    custom_objects={
        "minBert": minBert,
        "EncoderLayer": EncoderLayer,
        "SelfAttention": SelfAttention,
        "FeedForward": FeedForward,
        "BaseAttention" : BaseAttention,
        "PositionalEmbedding" : PositionalEmbedding
    }
)

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

# Load SNLI dataset
dataset = load_dataset("1-800-SHARED-TASKS/SNLI-NLI")

In [ ]:
# Tạo dữ liệu đầu vào (anchors, positives, negatives)
def prepare_data(dataset, tokenizer, batch_size=32, max_length=256):
    anchors = []
    positives = []
    negatives = []

    print(dataset[0])

    step = 1

    for i in range(0, len(dataset) - 2, step + 1):  # bước nhảy ở cuối mỗi nhóm (i+3)
        sample_1 = dataset[i]         # Phần tử đầu tiên
        sample_2 = dataset[i + 1]     # Phần tử thứ hai
        sample_3 = dataset[i + 2]     # Phần tử thứ ba

        group = [sample_1, sample_2, sample_3]
        for sample in group:
          if sample['label'] == 0:
              anchors.append(sample['premise'])
              positives.append(sample['hypothesis'])
          elif sample['label'] == 2:
              negatives.append(sample['hypothesis'])

    min_len = min(len(anchors), len(positives), len(negatives))
    anchors = anchors[:min_len]
    positives = positives[:min_len]
    negatives = negatives[:min_len]

    print("Tổng số anchors:", len(anchors))
    print("Tổng số positives:", len(positives))
    print("Tổng số negatives:", len(negatives))
    # Kiểm tra xem có giá trị None hay không trong dữ liệu đầu vào
    if any(val is None for val in anchors + positives + negatives):
        print("Cảnh báo: Tìm thấy None trong các dữ liệu đầu vào!")

    # Tiến hành tokenization
    def tokenize_texts(texts):
        encodings = tokenizer(texts, padding='max_length', truncation=True, max_length=max_length, return_tensors="tf")
        return encodings

    anchor_encodings = tokenize_texts(anchors)
    positive_encodings = tokenize_texts(positives)
    negative_encodings = tokenize_texts(negatives)

    # Kiểm tra xem tokenization có trả về None không
    if anchor_encodings is None or positive_encodings is None or negative_encodings is None:
        print("Cảnh báo: Tokenizer không thành công với một số mẫu!")

    # Tạo tf.data.Dataset
    dataset = {
        'anchors': anchor_encodings['input_ids'],
        'positives': positive_encodings['input_ids'],
        'negatives': negative_encodings['input_ids']
    }

    return dataset



In [ ]:
train_dataset = prepare_data(dataset['train'].select(range(200000)), tokenizer)

In [ ]:
def create_tf_dataset(train_data, batch_size=32):
    # Chuyển thành tf.data.Dataset
    dataset = tf.data.Dataset.from_tensor_slices((train_data['anchors'], train_data['positives'], train_data['negatives']))
    dataset = dataset.batch(batch_size)
    return dataset

# Tạo dataset cho huấn luyện
tf_train_dataset = create_tf_dataset(train_dataset)

In [ ]:
model = SupervisedContrastiveModel(min_bert_layer)

# Đặt các tham số huấn luyện
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5))

# Huấn luyện mô hình
model.fit(tf_train_dataset, epochs=3)

Epoch 1/3


/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'query' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'key' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:934: UserWarning: Layer 'value' (of type EinsumDense) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/ke

3118/3118 ━━━━━━━━━━━━━━━━━━━━ 1908s 599ms/step - loss: 2.2625
Epoch 2/3
3118/3118 ━━━━━━━━━━━━━━━━━━━━ 1901s 593ms/step - loss: 1.8796
Epoch 3/3
3118/3118 ━━━━━━━━━━━━━━━━━━━━ 1884s 593ms/step - loss: 1.7276


In [ ]:
min_bert_layer.save("min_bert_layer_after_nli.keras")